In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
data=pd.read_csv('Modified_SMSSpamCollection.csv')
data

,Label,Message
0,"""ham","Go until jurong point, crazy.. Available only ..."
1,"""ham","Ok lar... Joking wif u oni..."""
2,"""spam",Free entry in 2 a wkly comp to win FA Cup fina...
3,"""ham",U dun say so early hor... U c already then say...
4,"""ham","Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,"""spam",This is the 2nd time we have tried 2 contact u...
5570,"""ham","Will ü b going to esplanade fr home?"""
5571,"""ham","Pity, * was in mood for that. So...any other s..."
5572,"""ham",The guy did some bitching but I acted like i'd...


In [31]:
print(data.columns)

Index(['Label', 'Message'], dtype='object')


In [32]:
data['Label']=data['Label'].str[1:]
data['Message']=data['Message'].str[0:-1]
data

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,ham,Will ü b going to esplanade fr home?
5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,ham,The guy did some bitching but I acted like i'd...


In [33]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
Lemmatizer=WordNetLemmatizer()

In [34]:
data['Label']=data['Label'].map({'spam':0,'ham':1})
data

,Label,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,0,This is the 2nd time we have tried 2 contact u...
5570,1,Will ü b going to esplanade fr home?
5571,1,"Pity, * was in mood for that. So...any other s..."
5572,1,The guy did some bitching but I acted like i'd...


In [35]:
for i in range(data.shape[0]):
    document=data.iloc[i,1]
    words=document.split()
    words=[Lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    document=" ".join(words)
    data.iloc[i,1]=document
data    

,Label,Message
0,1,"Go jurong point, crazy.. Available bugis n gre..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry 2 wkly comp win FA Cup final tkts 2...
3,1,U dun say early hor... U c already say...
4,1,"Nah I think go usf, life around though"
...,...,...
5569,0,This 2nd time tried 2 contact u. U £750 Pound ...
5570,1,Will ü b going esplanade fr home?
5571,1,"Pity, * mood that. So...any suggestions?"
5572,1,The guy bitching I acted like i'd interested b...


In [64]:
X=data[['Message']]
y=data[['Label']]

In [65]:
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [66]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train

,Message
1350,FREE2DAY sexy St George's Day pic Jordan!Txt P...
5544,Armand say get as epsilon
1168,Lol I'm hot air balloon!
5551,"You know, wot people wear. T shirts, jumpers, ..."
5320,"Good morning, Love ... I go sleep wish great d..."
...,...
3772,"Hi, wlcome back, wonder got eaten lion somethi..."
5191,ree entry 2 weekly comp chance win ipod. Txt P...
5226,"""""OH FUCK. JUSWOKE UP IN A BED ON A BOATIN THE..."
5390,NOT MUCH NO FIGHTS. IT WAS A GOOD NITE!!


In [67]:
print(X_train.columns)

Index(['Message'], dtype='object')


In [68]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [69]:
cv=CountVectorizer(max_features=1000,ngram_range=(1,2))
tfidf=TfidfVectorizer(max_features=1000,ngram_range=(1,2))

In [70]:
X_train=cv.fit_transform(X_train['Message']).toarray()
X_test=cv.transform(X_test['Message']).toarray()

In [71]:
model=MultinomialNB()
model.fit(X_train,y_train)
model

C:\Users\mahap\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [72]:
y_pred=model.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [73]:
print(f"The accuracy score for spam ham classification by BOW is ",accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

The accuracy score for spam ham classification by BOW is  0.979372197309417
[[151  10]
 [ 13 941]]
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       161
           1       0.99      0.99      0.99       954

    accuracy                           0.98      1115
   macro avg       0.96      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [74]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [75]:
X_train=tfidf.fit_transform(X_train['Message']).toarray()
X_test=tfidf.transform(X_test['Message']).toarray()
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [76]:
model.fit(X_train,y_train)
model

C:\Users\mahap\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [77]:
y_pred=model.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [78]:
print(f"The accuracy score for spam ham classification by TFIDF is ",accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

The accuracy score for spam ham classification by TFIDF is  0.9820627802690582
[[143  18]
 [  2 952]]
              precision    recall  f1-score   support

           0       0.99      0.89      0.93       161
           1       0.98      1.00      0.99       954

    accuracy                           0.98      1115
   macro avg       0.98      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115

